In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 

In [5]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [2]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']

In [7]:
import joblib 


,recall_score,precision_score,f1_score
class,,,
AGN,0.974530,0.968465,0.971488
CV,0.536145,0.593333,0.563291
HMXB,0.905080,0.916103,0.910558
LMXB,0.811189,0.943089,0.872180
PULSAR,0.455446,0.418182,0.436019
STAR,0.956272,0.959712,0.957989
ULX,0.706161,0.699531,0.702830
YSO,0.951262,0.927844,0.939407


In [ ]:
all_feat = {
    'AGN' : {
        'precision' : [] , 
    'recall' : [],
    'f1_score' : []
    }
}
for i in range(5):
    model = joblib.load('models/all_feat_v1_0.pkl')
    all_feat['precision'].append(model.result['class_scores']['precision_score']['AGN']
    

In [13]:
model.result['class_scores']['precision_score']['AGN']

0.9684647302904564